# visualizer.ipynb

Updated 2026-04-19 11:14

This Jupyter notebook visualizes detections of coconut palms in images using data from an SQLite database.

Each detection is visualized using a bound box and polygon overlay applied to the original image.

In [1]:
import sqlite3
import cv2
import pandas as pd
import numpy as np
import os
import shutil
import roadside as rs
from icecream import ic
import textwrap

In [2]:
# Set variables for this run
database_path = 'sam3_detections.sqlite3'
sql = """
SELECT i.image_path, i.image_id, d.*
FROM images as i
INNER JOIN detections as d
on i.image_id = d.image_id
order by image_path, confidence DESC, detection_id;
"""
annotated_image_dir = 'annotated_images'
delete_annotated_image_dir_before_run = True

In [3]:
def annotate_image_with_polygon_and_rectangle(original_image_path, x_min, y_min, x_max, y_max, polygon_points, color):
    # Load original image
    img_original = cv2.imread(original_image_path).copy()
    # ic(img_original.shape)
    # ic(type(polygon_points))
    # ic(polygon_points.dtype)
    # ic(polygon_points.shape)
    
    # Overlay detected object with a semitransparent polygon mask
    img_overlay = np.zeros_like(img_original)
    alpha = 0.4  # Transparency factor (0.0 = transparent, 1.0 = opaque)
    img_overlay = cv2.fillPoly(img_overlay, [polygon_points], color)
    img_annotated = cv2.addWeighted(img_overlay, alpha, img_original, 1 - alpha, 0)

    # Draw rectangle
    start_point = (x_min, y_min)
    end_point = (x_max, y_max)
    thickness = 1
    cv2.rectangle(img_annotated, start_point, end_point, color, thickness)
    
    # Extract the region inside the rectangle
    roi = img_annotated[y_min:y_max+1, x_min:x_max+1]
    
    return img_annotated, roi

# Main

In [4]:
if os.path.exists(annotated_image_dir) and delete_annotated_image_dir_before_run:
    shutil.rmtree(annotated_image_dir)

# Ensure existence of output directory
os.makedirs(annotated_image_dir, exist_ok=True)

In [5]:
# Read detections from the database
conn = sqlite3.connect(database_path)
df = pd.read_sql_query(sql, conn)
conn.close()
df

,image_path,image_id,detection_id,image_id,class_id,poly_wkt,x_min,y_min,x_max,y_max,confidence
0,08hs-palms-03-zglw-superJumbo.webp,2,27,2,0,"POLYGON ((486 679, 486 684, 485 685, 485 688, ...",354,683,593,2068,0.928711
1,08hs-palms-03-zglw-superJumbo.webp,2,26,2,0,"POLYGON ((1035 1274, 1035 1276, 1034 1277, 103...",701,1273,1103,2066,0.920898
2,20251129_152106.jpg,1,17,1,0,"POLYGON ((1154 233, 1153 234, 1151 234, 1150 2...",939,232,1329,997,0.842285
3,20251129_152106.jpg,1,6,1,0,"POLYGON ((1681 715, 1680 716, 1680 717, 1675 7...",1592,715,1778,905,0.771484
4,20251129_152106.jpg,1,10,1,0,"POLYGON ((1013 829, 1008 834, 1006 834, 1003 8...",963,828,1055,985,0.763672
5,20251129_152106.jpg,1,4,1,0,"POLYGON ((902 863, 902 865, 901 866, 901 867, ...",817,790,938,971,0.743164
6,20251129_152106.jpg,1,13,1,0,"POLYGON ((568 339, 566 341, 566 346, 567 347, ...",341,338,826,1018,0.729980
7,20251129_152106.jpg,1,3,1,0,"POLYGON ((538 795, 537 796, 535 796, 534 797, ...",491,794,561,961,0.698730
8,20251129_152106.jpg,1,21,1,0,"POLYGON ((1101 963, 1101 963, 1103 962, 1104 9...",1065,684,1217,973,0.690918
9,20251129_152106.jpg,1,8,1,0,"POLYGON ((1130 942, 1130 945, 1129 946, 1129 9...",1046,801,1136,977,0.677734


In [6]:
# Annotate images based on detections (one image per detection)
# Images are saved as new files with -XX.jpg suffix where XX is the order returned by the SQL query.
for i, r in df.iterrows():
    original_image_path = r['image_path']
    detection_id = r['detection_id']
    x_min = int(r['x_min'])
    y_min = int(r['y_min'])
    x_max = int(r['x_max'])
    y_max = int(r['y_max'])
    polygon_points_str = r['poly_wkt']
    # polygon_points = np.array(eval(polygon_points_str), np.int32)          
    # polygon_points = [polygon_points.reshape((-1, 1, 2))]
    polygon_points = rs.conv_poly_from_wkt_to_array(polygon_points_str)
    color = (0, 0, 255)  # Red color in BGR
    annotated_image_path = f'{annotated_image_dir}/{original_image_path}-{detection_id:02d}.jpg' 
    roi_image_path = f'{annotated_image_dir}/roi-{original_image_path}-{detection_id:02d}.jpg' 
    print(f'Annotating image: {annotated_image_path}')   
    img_annotated, roi = annotate_image_with_polygon_and_rectangle(original_image_path, x_min, y_min, x_max, y_max, polygon_points, color)
    print(f'saving {annotated_image_path}')
    print(cv2.imwrite(annotated_image_path, img_annotated))
    print(f'saving {roi_image_path}')
    print(cv2.imwrite(roi_image_path, roi))
    
    # Write .txt file describing the annotated image. May be used in a caption later.
    txt_path = f'{annotated_image_path}.txt'
    with open(txt_path, 'w') as f:
        f.write(f'{original_image_path}   detection_id: {r["detection_id"]}   confidence: {r["confidence"]:.3f} \n')
    
print('FINISHED')

Annotating image: annotated_images/08hs-palms-03-zglw-superJumbo.webp-27.jpg
saving annotated_images/08hs-palms-03-zglw-superJumbo.webp-27.jpg
True
saving annotated_images/roi-08hs-palms-03-zglw-superJumbo.webp-27.jpg
True
Annotating image: annotated_images/08hs-palms-03-zglw-superJumbo.webp-26.jpg
saving annotated_images/08hs-palms-03-zglw-superJumbo.webp-26.jpg
True
saving annotated_images/roi-08hs-palms-03-zglw-superJumbo.webp-26.jpg
True
Annotating image: annotated_images/20251129_152106.jpg-17.jpg
saving annotated_images/20251129_152106.jpg-17.jpg
True
saving annotated_images/roi-20251129_152106.jpg-17.jpg
True
Annotating image: annotated_images/20251129_152106.jpg-06.jpg
saving annotated_images/20251129_152106.jpg-06.jpg
True
saving annotated_images/roi-20251129_152106.jpg-06.jpg
True
Annotating image: annotated_images/20251129_152106.jpg-10.jpg
saving annotated_images/20251129_152106.jpg-10.jpg
True
saving annotated_images/roi-20251129_152106.jpg-10.jpg
True
Annotating image: an

In [7]:
# Generate a markdown report

# NOTE: would be more efficient to do these calculations within the main loop


def generate_section(caption, annotated_image_path, roi_image_path):
    my_section = textwrap.dedent(f"""\
        ## {caption}

        ![]({annotated_image_path})

        ![]({roi_image_path})

        ### Attributes

        - [] accept
        - [] healthy
        - [] damaged
        - [] vcuts
        - [] dead
        - [] crowd
        - [] occluded
        - [] other_problem
        ---
    """)
    return my_section


s = ''
for i, r in df.iterrows():
    annotated_image_path = f'{annotated_image_dir}/{r['image_path']}-{r['detection_id']:02d}.jpg'
    roi_image_path = f'{annotated_image_dir}/roi-{r['image_path']}-{r['detection_id']:02d}.jpg'
    caption_path = f'{annotated_image_path}.txt'
    
    with open(caption_path) as f:
        caption = f.read()
    
    s += generate_section(caption, annotated_image_path, roi_image_path)

with open('report.md', 'w') as f:
    f.write(s)
    